In [ ]:
!pip install requests lxml beautifulsoup4 playwright google-cloud-pubsub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.3/321.3 KB 4.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.2/323.2 KB 8.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 KB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.7/173.7 KB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 28.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.1/223.1 KB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.5/297.5 KB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.1/83.1 KB 9.5 MB/s eta 0:00:00


In [ ]:
!playwright install

In [ ]:
!playwright install-deps

In [ ]:
import asyncio
import random
import json
import logging
import time
from datetime import datetime
from google.cloud import pubsub_v1
import hashlib
import requests
from bs4 import BeautifulSoup, BeautifulSoup as BS
from playwright.async_api import async_playwright

SITEMAP_URL = "https://static.theprotocol.it/sitemaps/CurrentOffers/SiteMapJobOffers1.xml"
CONCURRENCY = 5

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
DATA_FILENAME = f"protocol_data_{timestamp}.jsonl"
LOG_FILENAME = f"protocol_log_{timestamp}.log"

PROJECT_ID = "laborinsight-data"
TOPIC_ID = "jobs-raw"
publisher = pubsub_v1.PublisherClient()
topic_path = publisher.topic_path(PROJECT_ID, TOPIC_ID)

logger = logging.getLogger("protocol_scraper")
logger.setLevel(logging.INFO)

formatter = logging.Formatter("%(asctime)s [%(levelname)s] %(message)s")

console_handler = logging.StreamHandler()
console_handler.setFormatter(formatter)

file_handler = logging.FileHandler(LOG_FILENAME, encoding="utf-8")
file_handler.setFormatter(formatter)

logger.handlers.clear()
logger.addHandler(console_handler)
logger.addHandler(file_handler)

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/129.0.0.0 Safari/537.36"
    )
}


def parse_list(section):
    if not section:
        return None
    items = []
    for li in section.select("li"):
        t = li.get_text(strip=True)
        if t:
            items.append(t)
    return items if items else None


def extract_title(soup: BeautifulSoup):
    tag = soup.find("h1")
    return tag.get_text(strip=True) if tag else None


def extract_company(soup: BeautifulSoup):
    tag = soup.select_one('[data-test="text-offerEmployer"]')
    if not tag:
        return None

    span = tag.find("span")
    if span:
        span.extract()

    return tag.get_text(strip=True)

def extract_salary(soup: BeautifulSoup):
    container = soup.select_one('[data-test="text-salary-value"]')
    if not container:
        return None

    if container.get("data-is-secondary") == "true":
        return container.get_text(strip=True)

    salary_value = container.select_one('[data-test="text-contractSalary"]')
    salary_units = container.select_one('[data-test="text-contractUnits"]')
    salary_time = container.select_one('[data-test="text-contractTimeUnits"]')

    parts = []
    if salary_value:
        parts.append(salary_value.get_text(" ", strip=True))
    if salary_units:
        parts.append(salary_units.get_text(" ", strip=True))
    if salary_time:
        parts.append(salary_time.get_text(" ", strip=True))

    return " ".join(parts) if parts else None


def extract_location(soup: BeautifulSoup):
    loc = soup.select_one('[data-test="text-primaryLocation"]')
    if loc:
        return loc.get_text(strip=True)

    loc2 = soup.select_one('[data-test="text-currentLocation1"]')
    if loc2:
        return loc2.get_text(strip=True)

    return None


def extract_level(soup: BeautifulSoup):
    tag = soup.select_one('[data-test="content-positionLevels"]')
    if not tag:
        return None

    txt = tag.get_text(strip=True).lower()
    if "/" in txt:
        return [x.strip() for x in txt.split("/")]
    return txt


def extract_requirements(soup: BeautifulSoup):
    result = {"expected": [], "optional": []}

    expected = soup.select_one('[data-test="section-requirements-expected"]')
    optional = soup.select_one('[data-test="section-requirements-optional"]')

    if expected:
        result["expected"] = parse_list(expected) or []
    if optional:
        result["optional"] = parse_list(optional) or []

    return result


def extract_responsibilities(soup: BeautifulSoup):
    section = soup.select_one('[data-test="section-responsibilities"]')
    return parse_list(section)


def extract_offered_and_benefits(soup: BeautifulSoup):
    offered = soup.select_one('[data-test="section-offered"]')
    benefits = soup.select_one('[data-test="section-benefits"]')

    return {
        "offered": parse_list(offered),
        "benefits": parse_list(benefits),
    }


def extract_technologies(soup: BeautifulSoup):
    sections = []

    for h3 in soup.find_all("h3"):
        title = h3.get_text(strip=True).lower()
        container = h3.find_next("div")
        if not container:
            continue

        chips = container.select('[data-test="chip-technology"]')

        values = []
        for c in chips:
            if c.get("title"):
                values.append(c["title"].strip())
            else:
                t = c.get_text(strip=True)
                if t:
                    values.append(t)

        if values:
            sections.append(values)

    required = sections[0] if len(sections) > 0 else []
    nice = sections[1] if len(sections) > 1 else []

    return required, nice


def parse_offer(html: str, url: str):
    soup = BeautifulSoup(html, "lxml")

    title = extract_title(soup)
    company = extract_company(soup)
    location = extract_location(soup)
    level = extract_level(soup)
    salary = extract_salary(soup)
    tech_required, tech_nice = extract_technologies(soup)
    requirements = extract_requirements(soup)
    responsibilities = extract_responsibilities(soup)
    offered = extract_offered_and_benefits(soup)

    return {
        "title": title,
        "company": company,
        "location": location,
        "level": level,
        "salary": salary,
        "tech_required": tech_required,
        "tech_nice_to_have": tech_nice,
        "requirements": requirements,
        "responsibilities": responsibilities,
        "offered": offered,
        "url": url,
    }


async def fetch_offer(page, url: str, retries: int = 3):
    for attempt in range(retries):
        try:
            await asyncio.sleep(random.uniform(0.2, 1.1))
            await page.goto(url, timeout=60000, wait_until="domcontentloaded")
            html = await page.content()
            return parse_offer(html, url)
        except Exception as e:
            if attempt == retries - 1:
                logger.error(f"Fetch error for {url}: {e}")
                return {"url": url, "error": str(e)}
            await asyncio.sleep(1 + attempt)


async def scrape_all(urls):
    results = []

    async with async_playwright() as pw:
        browser = await pw.chromium.launch(
            headless=True,
            args=["--disable-blink-features=AutomationControlled"],
        )

        semaphore = asyncio.Semaphore(CONCURRENCY)

        async def worker(url, idx, total):
            async with semaphore:
                page = await browser.new_page(user_agent=HEADERS["User-Agent"])
                data = await fetch_offer(page, url)
                await page.close()
                logger.info(
                    f"[{idx}/{total}] {data.get('title')} | "
                    f"{data.get('company')} | {data.get('location')} | {data.get('url')}"
                )

                return data

        tasks = [
            asyncio.create_task(worker(url, i + 1, len(urls)))
            for i, url in enumerate(urls)
        ]

        for task in asyncio.as_completed(tasks):
            results.append(await task)

        await browser.close()

    return results


async def load_sitemap(url: str):
    logger.info("Fetching sitemap (requests)...")
    try:
        r = requests.get(url, headers=HEADERS, timeout=30)
        if r.status_code == 403:
            logger.error("Got 403 Forbidden when fetching sitemap via requests. Falling back to Playwright...")
            raise requests.HTTPError("403 from requests")
        r.raise_for_status()
        xml = BS(r.text, "xml")
        urls = [loc.get_text() for loc in xml.find_all("loc")]
        logger.info(f"Found {len(urls)} URLs in sitemap (requests)")
        return urls
    except Exception as e:
        logger.error(f"Error fetching sitemap via requests: {e}")
        logger.info("Trying to fetch sitemap via Playwright...")

    # Fallback: Playwright udająca przeglądarkę
    try:
        async with async_playwright() as pw:
            browser = await pw.chromium.launch(headless=True)
            page = await browser.new_page(user_agent=HEADERS["User-Agent"])
            await page.goto(url, wait_until="domcontentloaded", timeout=60000)
            xml_text = await page.content()
            await browser.close()

        # page.content() da Ci <html>...</html> z XML-em w środku, ale <loc> będą
        soup = BS(xml_text, "xml")
        urls = [loc.get_text() for loc in soup.find_all("loc")]
        logger.info(f"Found {len(urls)} URLs in sitemap (Playwright)")
        return urls
    except Exception as e:
        logger.error(f"Failed to fetch sitemap even via Playwright: {e}")
        return []


def make_fingerprint(offer: dict) -> str:
    basis = f"{offer.get('title','')}|{offer.get('company','')}|{offer.get('location','')}|{offer.get('url','')}"
    return hashlib.sha1(basis.encode("utf-8")).hexdigest()

async def main():
    start = time.perf_counter()

    urls = await load_sitemap(SITEMAP_URL)
    # urls = urls[:10]  # do testów

    if not urls:
        logger.error("No URLs fetched from sitemap. Exiting without scraping.")
        return

    logger.info(f"Starting scraping of {len(urls)} offers...")

    results = await scrape_all(urls)

    now = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")
    published = 0

    for offer in results:
        if not offer.get("title") or "error" in offer:
            continue 

        msg = {
            "source": "theprotocol",
            "payload": offer,
            "ingested_at": now,
            "fingerprint": make_fingerprint(offer),
        }

        data_bytes = json.dumps(msg, ensure_ascii=False).encode("utf-8")
        publisher.publish(topic_path, data=data_bytes)
        published += 1

    elapsed = time.perf_counter() - start
    logger.info(f"Published {published} messages to Pub/Sub topic {TOPIC_ID}")
    logger.info(f"Exec time: {elapsed:.2f} s (~{elapsed/60:.1f} min)")

# if __name__ == "__main__":
#     asyncio.run(main())



In [7]:
await main()

2025-11-16 13:39:47,347 [INFO] Fetching sitemap...
2025-11-16 13:39:49,178 [INFO] Found 6112 URLs in sitemap
2025-11-16 13:39:49,180 [INFO] Starting scraping of 10 offers...
2025-11-16 13:39:52,941 [INFO] [4/10] Data analyst - product owner | Santander Bank Polska | Warszawa, Wola | https://theprotocol.it/szczegoly/praca/data-analyst---product-owner-warszawa-aleja-jana-pawla-ii-17,oferta,12a60000-fe33-2e51-c7b1-08de02813576
2025-11-16 13:39:54,094 [INFO] [5/10] Data analyst | Santander Bank Polska | Warszawa, mazowieckie | https://theprotocol.it/szczegoly/praca/data-analyst-warszawa,oferta,08dd0000-54c1-3a03-1cc4-08de028153ef
2025-11-16 13:39:54,512 [INFO] [2/10] Kurs na Kontrolera Ruchu Lotniczego | Polska Agencja Żeglugi Powietrznej | Warszawa, Włochy | https://theprotocol.it/szczegoly/praca/kurs-na-kontrolera-ruchu-lotniczego-warszawa-wiezowa-8,oferta,ed720000-f460-1a83-c9f2-08de01872a90
2025-11-16 13:39:54,783 [INFO] [3/10] Senior Java Developer API Management (F/M) | PEOPLEVIBE SP